In [104]:
import requests
import bs4
from glob import glob
import youtube_dl
import librosa

In [14]:
genres = {
    'country': 'https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S', 
    'jazz': 'https://www.youtube.com/playlist?list=PL8F6B0753B2CCA128', 
    'hip_hop': 'https://www.youtube.com/playlist?list=PLAPo1R_GVX4IZGbDvUH60bOwIOnZplZzM', 
    'classical': 'https://www.youtube.com/playlist?list=PLRb-5mC4V_Lop8KLXqSqMv4_mqw5M9jjW', 
    'metal': 'https://www.youtube.com/playlist?list=PLfY-m4YMsF-OM1zG80pMguej_Ufm8t0VC', 
    'electronic': 'https://www.youtube.com/playlist?list=PLDDAxmBan0BKeIxuYWjMPBWGXDqNRaW5S'
}

In [101]:
url = "https://www.youtube.com/watch?v=TI8nSJ9-C3c"
ydl_opts = {
    'format': 'bestaudio/best',
    'ignoreerrors': True,
    'outtmpl': '%(title)s.%(ext)s',
    'playlistend': 1,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
        'preferredquality': '192',
    }],
}
# with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#     ydl.download(['https://www.youtube.com/watch?v=zXDAYlhdkyg&list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S'])
ydl = youtube_dl.YoutubeDL(ydl_opts)

In [11]:
# youtube_dl.YoutubeDL?

In [56]:
response = requests.get('https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S')

In [57]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.select()

In [78]:
link_sfxs = ([a['href'] for a in soup.find_all('a', class_='pl-video-title-link')])

In [102]:
url = 'https://www.youtube.com' + link_sfxs[0]
ydl.download([url])

[youtube:playlist] Downloading playlist PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S - add --no-playlist to just download video ZEWGyyLiqY4
[youtube:playlist] PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S: Downloading webpage
[download] Downloading playlist: Best Country Songs of All Time - Top Country Music Videos
[youtube:playlist] playlist Best Country Songs of All Time - Top Country Music Videos: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube] ZEWGyyLiqY4: Downloading webpage
[youtube] ZEWGyyLiqY4: Downloading video info webpage
[download] Destination: Blake Shelton - God's Country (Official Music Video).webm
[download] 100% of 3.29MiB in 00:01
[ffmpeg] Destination: Blake Shelton - God's Country (Official Music Video).m4a
Deleting original file Blake Shelton - God's Country (Official Music Video).webm (pass -k to keep)
[download] Finished downloading playlist: Best Country Songs of All Time - Top Country Music Videos


0

## Broad overview
For each genre
1. Get urls (_beatifulSoup_)
For each url
 1. Download - url to m4a(_youtube_dl_)
 3. MIR - m4a to sonic features(_librosa_)
 5. Remove *.m4a

## Detailed
for each genre:
1. collect urls (_beatifulSoup_)  
for each url
 1. __Download__ - url to m4a(_youtube_dl_)
 2. __Assign filepath__(_glob_)
 3. __MIR__ - m4a to array(_librosa_)
 4. __MIR__ - array to individual segments  
 5. __For each segment__
    1. __MIR__ - array to beats, mfccs, zcr...  
    2. Append to __db__(_sql/csv/json??_)
    
 5. Remove *.m4a

EDGE CASE - error handling

In [106]:
glob('*.m4a')

["Blake Shelton - God's Country (Official Music Video).m4a"]

In [107]:
ts, sr = librosa.load(glob('*.m4a')[0])

In [ ]:
ts.